In [1]:
from __future__ import print_function
from packaging.version import parse as Version
from platform import python_version

OK = '\x1b[42m[ OK ]\x1b[0m'
FAIL = "\x1b[41m[FAIL]\x1b[0m"

try:
    import importlib
except ImportError:
    print(FAIL, "Python version 3.11 is required,"
                " but %s is installed." % sys.version)

def import_version(pkg, min_ver, fail_msg=""):
    mod = None
    try:
        mod = importlib.import_module(pkg)
        if pkg in {'PIL'}:
            ver = mod.VERSION
        else:
            ver = mod.__version__
        if Version(ver) == Version(min_ver):
            print(OK, "%s version %s is installed."
                  % (lib, min_ver))
        else:
            print(FAIL, "%s version %s is required, but %s installed."
                  % (lib, min_ver, ver))    
    except ImportError:
        print(FAIL, '%s not installed. %s' % (pkg, fail_msg))
    return mod


# first check the python version
pyversion = Version(python_version())

if pyversion >= Version("3.11.4"):
    print(OK, "Python version is %s" % pyversion)
elif pyversion < Version("3.11"):
    print(FAIL, "Python version 3.11 is required,"
                " but %s is installed." % pyversion)
else:
    print(FAIL, "Unknown Python version: %s" % pyversion)

    
print()
requirements = {'numpy': "1.24.4", 'matplotlib': "3.7.2",'sklearn': "1.3.0", 
                'pandas': "2.0.3",'xgboost': "1.7.6", 'shap': "0.42.1", 'seaborn': "0.12.2"}

# now the dependencies
for lib, required_version in list(requirements.items()):
    import_version(lib, required_version)

[ OK ] Python version is 3.11.4

[ OK ] numpy version 1.24.4 is installed.
[ OK ] matplotlib version 3.7.2 is installed.
[ OK ] sklearn version 1.3.0 is installed.
[ OK ] pandas version 2.0.3 is installed.
[ OK ] xgboost version 1.7.6 is installed.


Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


[ OK ] shap version 0.42.1 is installed.
[ OK ] seaborn version 0.12.2 is installed.


In [2]:
# Data Handling and Array Manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

# Preprocessing and Pipeline Tools
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

# Machine Learning Models
from sklearn.ensemble import RandomForestClassifier

# Model Selection and Evaluation
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    precision_score, recall_score, fbeta_score, auc,
    precision_recall_curve, average_precision_score
)

# Miscellaneous
import warnings
warnings.filterwarnings('ignore')
import pickle

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

df = pd.read_csv('data/CVD_cleaned.csv')
df.head()
ftrs = df.columns

random_state = 56

X = df.drop(labels=['Heart_Disease'], axis=1)
y = df['Heart_Disease']

label_mapping = {'No': 0, 'Yes': 1}
y = y.map(label_mapping)

X_subset, _, y_subset, _ = train_test_split(X, y, test_size=0.99, stratify=y, random_state=56)

print("Full Set")
print('X:',X.shape)
print('y:',y.shape)
#print("Subset Set")
#print('X_subset:',X_subset.shape)
#print('y_subset:',y_subset.shape)

cat_ftrs = ['Checkup','Exercise','Skin_Cancer','Other_Cancer', 'Depression', 'Diabetes', 'Arthritis', 'Sex','Smoking_History']
ordinal_ftrs = ['General_Health','Age_Category',]
ordinal_cats = [['Poor','Fair','Good','Very Good','Excellent'],\
               ['18-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70-74','75-79','80+']]
num_ftrs = ['Height_(cm)', 'Weight_(kg)', 'BMI', 'Alcohol_Consumption', 'Fruit_Consumption',
       'Green_Vegetables_Consumption', 'FriedPotato_Consumption']

# one-hot encoder
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse_output=False,handle_unknown='ignore'))])

# ordinal encoder
ordinal_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder(categories = ordinal_cats))])

# standard scaler
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

# collect the transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_ftrs),
        ('cat', categorical_transformer, cat_ftrs),
        ('ord', ordinal_transformer, ordinal_ftrs)])
final_scaler = StandardScaler()
prep = Pipeline(steps=[('preprocessor', preprocessor)])

Full Set
X: (308854, 18)
y: (308854,)


### baseline accuracy

In [4]:
N, Y = df['Heart_Disease'].value_counts()
baseline_accuracy = Y/(N+Y)
print('baseline accuracy score for the entire dataset is', baseline_accuracy)

baseline accuracy score for the entire dataset is 0.08085049894124732


### Random Forest Classifier

In [5]:
# Assuming 'prep' is your preprocessing pipeline and 'X_subset', 'y_subset' are your dataset
nr_states = 5
rf_scores = []
rf_models = []
rf_test_sets = []

for i in range(nr_states):
    print('Random state', i + 1)

    # Splitting the dataset
    X_train, X_other, y_train, y_other = train_test_split(X, y, train_size=0.6, stratify=y, random_state=56*i)
    X_val, X_test, y_val, y_test = train_test_split(X_other, y_other, train_size=0.5, stratify=y_other, random_state=56*i)
    print(X_test.shape)
    # Create a pipeline with preprocessing and RandomForestClassifier
    pipe = make_pipeline(prep, RandomForestClassifier(n_jobs=-1, random_state=56*i))

    # Define the parameter grid
    param_grid = {
        'randomforestclassifier__max_depth': [1, 3, 10, 30, 100],
        'randomforestclassifier__max_features': [0.25, 0.5, 0.75, 1.0],
        'randomforestclassifier__class_weight': [None, 'balanced', 'balanced_subsample', {0: 1, 1: 10}, {0: 1, 1: 15}]
    }

    # GridSearchCV with the pipelines
    grid = GridSearchCV(pipe, param_grid, cv=3, scoring='recall', verbose=1, n_jobs=-1)
    grid.fit(X_train, y_train)

    # Best model
    best_model = grid.best_estimator_
    rf_models.append(best_model)
    #print(f"Best parameters for random state {i + 1}: {grid.best_params_}")

    # Evaluate on the train, validation, and test sets
    for X_save, y_save, dataset_name_save in [(X_train, y_train, 'train'), (X_val, y_val, 'validation'), (X_test, y_test, 'test')]:
        y_pred = best_model.predict(X_save)
        precision, recall, _ = precision_recall_curve(y_save, best_model.predict_proba(X_save)[:, 1])
        auc_pr = auc(recall, precision)
        score = {
            'dataset': dataset_name_save,
            'state': i + 1,
            'precision': precision_score(y_save, y_pred),
            'recall': recall_score(y_save, y_pred),
            'f2': fbeta_score(y_save, y_pred, beta=2),
            'auc-pr': auc_pr
        }
        rf_scores.append(score)
    
    # Save the test set for later use
    rf_test_sets.append({'X_test': X_test, 'y_test': y_test, 'state': i + 1})


Random state 1
(61771, 18)
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Random state 2
(61771, 18)
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Random state 3
(61771, 18)
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Random state 4
(61771, 18)
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Random state 5
(61771, 18)
Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [11]:
rf_score = pd.DataFrame(rf_scores)
rf_score
rf_train_recall = rf_score[rf_score['dataset'] == 'train']['recall']
rf_val_recall = rf_score[rf_score['dataset'] == 'validation']['recall']
rf_test_recall = rf_score[rf_score['dataset'] == 'test']['recall']

print('train recall mean:',np.mean(rf_train_recall))
print('validation recall mean:',np.mean(rf_val_recall))
print('test recall mean:',np.mean(rf_test_recall))
print('test recall standard deviation:',np.std(rf_test_recall))
print(round((np.mean(rf_test_recall)-baseline_accuracy)/np.std(rf_test_recall),4),'standard deviations above the baseline')

train recall mean: 0.9256891143295736
validation recall mean: 0.9242290748898678
test recall mean: 0.9252302763315979
test recall standard deviation: 0.02059482803737233
40.9996 standard deviations above the baseline


In [12]:
pd.DataFrame(rf_scores)
rf_test_sets[1]['X_test']

,General_Health,Checkup,Exercise,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
32795,Very Good,Within the past year,Yes,No,No,No,No,No,Male,55-59,167.0,76.00,27.25,No,2.0,5.0,12.0,1.0
265627,Very Good,Within the past 2 years,No,No,No,No,No,No,Male,45-49,185.0,92.53,26.91,No,0.0,4.0,4.0,8.0
114129,Excellent,Within the past 5 years,Yes,No,No,No,No,No,Male,25-29,178.0,61.23,19.37,No,8.0,12.0,8.0,4.0
61434,Excellent,Within the past year,Yes,No,No,No,No,No,Male,55-59,180.0,90.72,27.89,Yes,4.0,8.0,16.0,12.0
230570,Excellent,Within the past year,Yes,No,No,No,No,No,Female,18-24,168.0,54.43,19.37,No,10.0,12.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75070,Very Good,Within the past year,Yes,No,No,No,No,No,Female,65-69,160.0,52.16,20.37,No,2.0,4.0,9.0,3.0
154028,Very Good,Within the past year,Yes,Yes,Yes,No,No,Yes,Female,80+,157.0,58.97,23.78,No,0.0,30.0,8.0,4.0
136888,Excellent,Within the past 2 years,Yes,No,No,No,No,No,Male,55-59,168.0,70.76,25.18,No,0.0,30.0,4.0,1.0
53478,Good,Within the past year,Yes,No,No,No,No,Yes,Female,65-69,155.0,77.11,32.12,No,0.0,30.0,8.0,4.0


### Save for later use

In [13]:
import pickle
with open('rf_results.pkl', 'wb') as file:
    pickle.dump({'scores': rf_scores, 'models': rf_models, 'test_sets': rf_test_sets}, file)

In [14]:
with open('rf_results.pkl', 'rb') as file:
    data = pickle.load(file)
    rf_scores = data['scores']
    rf_models = data['models']
    rf_test_sets = data['test_sets']

## Correct Version

In [22]:
from sklearn.model_selection import PredefinedSplit
# Assuming 'prep' is your preprocessing pipeline and 'X_subset', 'y_subset' are your dataset
nr_states = 5
rf_scores = []
rf_models = []
rf_test_sets = []

for i in range(nr_states):
    print('Random state', i + 1)

    # Splitting the dataset
    X_train, X_other, y_train, y_other = train_test_split(X, y, train_size=0.6, stratify=y, random_state=56*i)
    X_val, X_test, y_val, y_test = train_test_split(X_other, y_other, train_size=0.5, stratify=y_other, random_state=56*i)
    column_names = X.columns

    # Combine training and validation sets
    X_combined = np.concatenate([X_train, X_val])
    X_combined = pd.DataFrame(X_combined, columns=column_names)
    y_combined = np.concatenate([y_train, y_val])
    split_index = [-1]*len(X_train) + [0]*len(X_val)  # -1 for training, 0 for validation
    fixedval = PredefinedSplit(test_fold=split_index)

    print(X_test.shape)

    # Create a pipeline with preprocessing and RandomForestClassifier
    pipe = make_pipeline(prep, RandomForestClassifier(n_jobs=-1, random_state=56*i))

    # Define the parameter grid
    param_grid = {
        'randomforestclassifier__max_depth': [1, 3, 10, 30, 100],
        'randomforestclassifier__max_features': [0.25, 0.5, 0.75, 1.0],
        'randomforestclassifier__class_weight': [None, 'balanced', 'balanced_subsample', {0: 1, 1: 10}, {0: 1, 1: 15}]
    }

    # GridSearchCV with the pipelines
    grid = GridSearchCV(pipe, param_grid, cv=fixedval, scoring='recall', verbose=1, n_jobs=-1)
    grid.fit(X_combined, y_combined)  # Fit on the combined training and validation sets

    # Best model
    best_model = grid.best_estimator_
    rf_models.append(best_model)

    # Evaluate on the train, validation, and test sets
    for X_save, y_save, dataset_name_save in [(X_train, y_train, 'train'), (X_val, y_val, 'validation'), (X_test, y_test, 'test')]:
        y_pred = best_model.predict(X_save)
        precision, recall, _ = precision_recall_curve(y_save, best_model.predict_proba(X_save)[:, 1])
        auc_pr = auc(recall, precision)
        score = {
            'dataset': dataset_name_save,
            'state': i + 1,
            'precision': precision_score(y_save, y_pred, zero_division=0),
            'recall': recall_score(y_save, y_pred),
            'f2': fbeta_score(y_save, y_pred, beta=2),
            'auc-pr': auc_pr
        }
        rf_scores.append(score)

    # Save the test set for later use
    rf_test_sets.append({'X_test': X_test, 'y_test': y_test, 'state': i + 1})


Random state 1
(61771, 18)
Fitting 1 folds for each of 100 candidates, totalling 100 fits
Random state 2
(61771, 18)
Fitting 1 folds for each of 100 candidates, totalling 100 fits
Random state 3
(61771, 18)
Fitting 1 folds for each of 100 candidates, totalling 100 fits
Random state 4
(61771, 18)
Fitting 1 folds for each of 100 candidates, totalling 100 fits
Random state 5
(61771, 18)
Fitting 1 folds for each of 100 candidates, totalling 100 fits


In [23]:
pd.DataFrame(rf_scores)
#rf_test_sets[1]['X_test']

,dataset,state,precision,recall,f2,auc-pr
0,train,1,0.144045,0.892345,0.437643,0.239416
1,validation,1,0.144944,0.894073,0.439633,0.242094
2,test,1,0.143526,0.889467,0.436132,0.241183
3,train,2,0.129805,0.934326,0.417188,0.239643
4,validation,2,0.128905,0.935322,0.415481,0.247448
5,test,2,0.129860,0.938326,0.417938,0.244925
6,train,3,0.137657,0.917974,0.430224,0.237719
7,validation,3,0.136302,0.912495,0.426613,0.236591
8,test,3,0.136520,0.914497,0.427390,0.232981
9,train,4,0.126581,0.935327,0.410621,0.228351


In [24]:
rf_score = pd.DataFrame(rf_scores)
rf_score
rf_train_recall = rf_score[rf_score['dataset'] == 'train']['recall']
rf_val_recall = rf_score[rf_score['dataset'] == 'validation']['recall']
rf_test_recall = rf_score[rf_score['dataset'] == 'test']['recall']

print('train recall mean:',np.mean(rf_train_recall))
print('validation recall mean:',np.mean(rf_val_recall))
print('test recall mean:',np.mean(rf_test_recall))
print('test recall standard deviation:',np.std(rf_test_recall))
print(round((np.mean(rf_test_recall)-baseline_accuracy)/np.std(rf_test_recall),4),'standard deviations above the baseline')

train recall mean: 0.9244210104785424
validation recall mean: 0.9234281137364839
test recall mean: 0.9242290748898678
test recall standard deviation: 0.01990295797470321
42.3745 standard deviations above the baseline


In [25]:
import pickle
with open('rf_results_final.pkl', 'wb') as file:
    pickle.dump({'scores': rf_scores, 'models': rf_models, 'test_sets': rf_test_sets}, file)

In [26]:
with open('rf_results_final.pkl', 'rb') as file:
    data = pickle.load(file)
    rf_scores = data['scores']
    rf_models = data['models']
    rf_test_sets = data['test_sets']